In [1]:
import pandas as pd
import pandas as pd
from bokeh.io import show, output_notebook, save
from bokeh.models import ColumnDataSource, Select, Legend
from bokeh.plotting import figure, show
from bokeh.models import CustomJS 
from bokeh.palettes import Plasma
from bokeh.layouts import row,column 
output_notebook()
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('data/crowdfunding.csv')
df['gender'] = [{'F': 'Female', 'M': 'Male', 'U': 'Unknown'}[g] for g in df['gender']]

grouped_df = df.groupby(['category', 'gender', 'age', 'device'], as_index = False).sum()

split_on_list = ['category', 'age', 'device', 'gender']
device_list = ['All'] + list(grouped_df['device'].unique())
gender_list = ['All'] + list(grouped_df['gender'].unique())
age_list = ['All'] + list(grouped_df['age'].unique())
category_list = ['All'] + list(grouped_df['category'].unique())
filtered_df = grouped_df[
        (grouped_df['device'] == 'android') & 
        (grouped_df['gender'] == 'Female') & 
        (grouped_df['age'] == '18-24')
    ]
filtered_df['x'] = range(len(filtered_df))
filtered_df['color'] = Plasma[5]
filtered_df['element'] = filtered_df['category']
original_DF = ColumnDataSource(data=grouped_df)
current_DF = ColumnDataSource(
    data=filtered_df)

dropdowns = {}
dropdowns['split_on'] = Select(options=split_on_list, value='category', title = 'Split on')  
dropdowns['category'] = Select(options=category_list, value='All', title = 'Category')  
dropdowns['device'] = Select(options=device_list, value='android', title = 'Device')  
dropdowns['gender'] = Select(options=gender_list, value='Female', title = 'Gender') 
dropdowns['age'] = Select(options=age_list, value='18-24', title = 'Age')

callback_code = """

sc.data['color'] = ['#0C0786', '#7C02A7', '#CA4678', '#F79341', '#EFF821']
sc.data['amount']=[]
sc.data['element']=[]
var amount = {}

for(var i = 0; i <= source.get_length(); i++){
    if (source.data['device'][i] != device.value && device.value != "All" ) continue;
    if (source.data['gender'][i] != gender.value && gender.value != "All") continue;
    if (source.data['age'][i] != age.value && age.value != "All") continue;
    if (source.data['category'][i] != category.value && category.value != "All") continue;
    if (source.data[split_on.value][i] in amount) {
        amount[source.data[split_on.value][i]] += source.data['amount'][i]
    }else{
        amount[source.data[split_on.value][i]] = source.data['amount'][i]
    }
}

sc.data['x']=[1,2,3,4,5]

for (const key in amount){
    if (typeof key !== 'undefined'){
        if (key != 'undefined'){
            sc.data['amount'].push(amount[key])
            sc.data['element'].push(key)
        }
    }
}
sc.change.emit();
"""


callback = CustomJS(args=dict(source=original_DF, sc=current_DF, **dropdowns), code=callback_code)

fig=figure(y_axis_label="amount", tools="hover", tooltips="@element: @amount")
fig.vbar(x='x',bottom=0, top="amount", width  = .5, source = current_DF, color = 'color', legend_field='element')
fig.add_layout(Legend(), 'left')
fig.xaxis.axis_label = None
fig.xaxis.visible = False
fig.xgrid.grid_line_color = None

# change just some things about the y-grid
fig.ygrid.grid_line_alpha = 1
fig.ygrid.grid_line_dash = [6, 4]

for dropdown in dropdowns:
    dropdowns[dropdown].js_on_change('value', callback) 

layout=row(column(dropdowns['category'], dropdowns['device'], dropdowns['gender'], dropdowns['age']), column(dropdowns['split_on'], fig)) # creating the layout
show(layout)

Loading BokehJS ...